In [28]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
#import matplotlib.pyplot as plt
import numpy as np

In [29]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [30]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=4,out_channels=96,kernel_size=11,stride=4,padding=0)

        self.conv2 = nn.Conv1d(in_channels=96,out_channels=96,kernel_size=1)
        
        self.conv3=nn.Conv1d(in_channels=96,out_channels=96,kernel_size=1)
        
        self.pool3 = nn.MaxPool1d(kernel_size=3,stride=2)

        self.conv4=nn.Conv1d(in_channels=96,out_channels=256,kernel_size=11,stride=4,padding=2)

        self.conv5=nn.Conv1d(in_channels=256,out_channels=256,kernel_size=1)

        self.conv6=nn.Conv1d(in_channels=256,out_channels=256,kernel_size=1)
        
        self.pool6 = nn.MaxPool1d(kernel_size=3,stride=2)

        self.conv7=nn.Conv1d(in_channels=256,out_channels=384,kernel_size=3,stride=1,padding=1)

        self.conv8=nn.Conv1d(in_channels=384,out_channels=384,kernel_size=1)

        self.conv9=nn.Conv1d(in_channels=384,out_channels=384,kernel_size=1)

        self.fc1 = nn.Linear(in_features=(384*1),out_features=2048)

        self.fc2 = nn.Linear(in_features=2048,out_features=2048)

        self.fc3 = nn.Linear(in_features=2048,out_features=2)

    def forward(self, x):
        x = F.relu(self.conv1(x))

        x = F.relu(self.conv2(x))

        x = self.pool3(F.relu(self.conv3(x)))
        x = F.dropout(x, 0.5)

        x = F.relu(self.conv4(x))

        x = F.relu(self.conv5(x))

        x = self.pool6(F.relu(self.conv6(x)))
        x = F.dropout(x, 0.5)

        x = F.relu(self.conv7(x))

        x = F.relu(self.conv8(x))

        x = F.relu(self.conv9(x))
        x = F.dropout(x, 0.5)

        x = torch.flatten(x, 1) # flatten all dimensions except batch
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)

        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)

        x = self.fc3(x)
        
        return x

In [31]:

# Define the path to your .txt file containing DNA sequences
file_path = "/users/saedunu/final_merge.txt"

# Read DNA sequences from the .txt file
with open(file_path, "r") as file:
    dna_data = [line.strip().split(":") for line in file]


In [32]:
first_pair = dna_data[0]

# Print the elements separately
print("ID:", first_pair[0])
print("DNA Sequence:", first_pair[1])

ID: 1
DNA Sequence: accctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaacccaaccctaaccctaaccctaaccctaaccctaaccctaacccctaaccctaaccctaaccctaaccctaacctaaccctaaccctaa


In [33]:
# Extract labels and sequences from the read data
labels, sequences = zip(*dna_data)
labels = np.array(labels, dtype=int)
sequences = np.array(sequences)

In [34]:
print(labels)
print(sequences)

[1 0 1 ... 0 1 0]
['accctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaacccaaccctaaccctaaccctaaccctaaccctaaccctaacccctaaccctaaccctaaccctaaccctaacctaaccctaaccctaa'
 'TGTGTCAGGGAAAGAATCAACCTGGGAGCCAAGCCCAGGAATCAGGATGACCAAGACATACTGCACAAGGAGGGAACAAACCCATCCAAGGACACTCAAGGACAAATCAAGCAAATGAATTTAAGGGAGACGTGCTCATGGTCTGCTTTGCTGCTCAGCATGGCTGGGAGGCACAGTGGA'
 'gcccaggggaggaggcgtggcgcaggcgcagagaggcgcgccgtgctgccgcaggcgcagagacacatgctagcgcgtccagggggtggaggcgtggcgcaggcgcagagacgcacgcctacgggcggggttggggggggcgtgtgttacaggagcaaagtcgcacggcgccgggctggg'
 ...
 'ctcttttgcctaatgatgcagttatggtcaatgagacccaaggatatgtctgttggctaaagataaaggatttcctccacaataaaaatgaatgtgcaagaagaaagctttgtccatacctttgaatggggctgtgagagggtagagcagctatcttgcaaccacaaggggacaaggctg'
 'CCGCGACTGCAGCAGGCCAACCTCGTGCTCGGGCCTCGCCCCTCCCCCACGCCTCTAGCTAGGCCCCGCCCAAGACCTGCCCCTCCGCGAGCCCCCGCCCACCTACAGCGCCCCCTGTGGCCACGCCCCTCACCGGCTCTCACGGGCGCAAGGGAACTAGATGTCGCTTCCTTTTGCCCA'
 'acgcccggctaattttttatatttttagtagagacagggtttcaccgtgttagcc

In [35]:

# Define the number of samples for training and testing
print(len(labels))
train_samples = int((80*(len(labels)))/100)
print(train_samples)
test_samples = len(labels)-train_samples
print(test_samples)

# Define the length of the DNA sequences
seq_len = len(sequences[0])

3000
2400
600


In [36]:
print(seq_len)

180


In [37]:
def one_hot_encoding(sequence):
    #sequence = sequence.tostring()
    #sequence = sequence.decode('utf-8')
    sequence = sequence.strip()
    mapping = {'A': 0, 'a':0, 'T': 1, 't':1, 'C': 2, 'c':2,'G': 3, 'g':3}
    # Define the size of the one-hot encoded vectors
    n = len(sequence)
    one_hot = np.zeros((4, n), dtype=np.int8)

    # Iterate through the sequence and set the corresponding element to 1
    for i, base in enumerate(sequence):
        #base = base.upper()  # Convert to uppercase to handle lowercase letters
        if base in mapping:
            one_hot[mapping[base], i] = 1
        else:
            # If the base is not 'A', 'T', 'G', or 'C', assign 0 to all positions
            one_hot[:, i] = 0

    return one_hot

In [38]:
list_l = []
for seq in sequences:
    # Call the one-hot encoding function on the current sequence
    encoded_seq = one_hot_encoding(seq)
    # Append the encoded sequence to the list
    list_l.append(encoded_seq)

# Convert the list to a NumPy array if needed
list_1 = np.array(list_l)


In [39]:
print(list_l[0])

[[1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1
  1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1
  1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1
  0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1
  1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1]
 [0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0
  0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0
  0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0
  0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0
  0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0]
 [0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0
  0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0
  0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 1 0 0 0
  1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 

In [40]:
random_train_data = np.array(list_1[:train_samples])

print(random_train_data)

print(random_train_data.shape)
print(random_train_data.dtype)

[[[1 0 0 ... 0 1 1]
  [0 0 0 ... 1 0 0]
  [0 1 1 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 1]
  [1 0 1 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 1 0 ... 1 1 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 1 1 ... 0 0 0]
  [1 0 0 ... 1 1 1]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 1 ... 1 1 0]
  [0 0 0 ... 0 0 0]
  [1 1 0 ... 0 0 1]]

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 1 0 0]
  [0 1 1 ... 0 1 0]
  [1 0 0 ... 0 0 0]]

 [[0 1 0 ... 0 0 0]
  [0 0 0 ... 1 0 0]
  [1 0 1 ... 0 0 1]
  [0 0 0 ... 0 1 0]]]
(2400, 4, 180)
int8


In [41]:
random_test_data = np.array(list_1[train_samples:train_samples+test_samples])

print(random_test_data[0])
print("=====")
print(random_test_data[0][0])

print(random_test_data.shape)
print(random_test_data.dtype)

[[0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0
  0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0
  0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0
  0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 1
  1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1
  0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 1 0 0
  1 1 0 0 0 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0
  0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0
  0 0 1 0 0 1 0 1 1 1 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 1 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0
  0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 0 0 

In [42]:
# Converting the data to tensor type and floating point type

tensor_train_data = torch.from_numpy(random_train_data).float()
tensor_test_data = torch.from_numpy(random_test_data).float()

print(tensor_train_data.shape)
print(tensor_train_data.dtype)

print(tensor_test_data.shape)
print(tensor_test_data.dtype)

torch.Size([2400, 4, 180])
torch.float32
torch.Size([600, 4, 180])
torch.float32


In [43]:
label_train = np.array(labels[:train_samples])
print(label_train)

label_test = np.array(labels[train_samples:train_samples+test_samples])
print(label_test)

print(label_train.dtype)
print(label_test.dtype)

label_train = torch.from_numpy(label_train)
label_test = torch.from_numpy(label_test)

print(label_train)
print(label_test)

print(label_train.shape)
print(label_test.shape)

print(label_train.dtype)
print(label_test.dtype)

[1 0 1 ... 0 1 0]
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 

In [44]:

# The most important class, a custom data loader, understand how it is working.

class data_class(Dataset):
    def __init__(self,data,label):
        #self.data=torch.tensor(data)
        #self.labels=torch.tensor(label)
        self.labels = torch.tensor(list(x for x in label))
        self.data=data
        #self.labels=label
        

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,id):
        data_set=self.data[id]
        labels=self.labels[id]

        return data_set,labels

In [45]:
# calling the data_class for the raw random data

train_data=data_class(tensor_train_data,label_train)
print(train_data)
print(len(train_data))
test_data=data_class(tensor_test_data,label_test)
print(test_data)
print(len(test_data))

2400
600


In [46]:
# Creating the data loader which is going to load the data to the AI model

train_dataloader=DataLoader(train_data,batch_size=3,shuffle=True)
print(train_dataloader)
print(len(train_dataloader))
test_dataloader=DataLoader(test_data,batch_size=3,shuffle=True)
print(test_dataloader)
print(len(test_dataloader))

800
200


In [47]:
model=Net().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [48]:
for data,label in train_dataloader:
    print(data)
    print(data.shape)
    print(label)
    print(len(train_dataloader))
    print(len(data[0][0]))
    break

tensor([[[0., 1., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 1.,  ..., 1., 0., 1.]],

        [[0., 0., 0.,  ..., 1., 1., 0.],
         [0., 1., 0.,  ..., 0., 0., 1.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 1.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]]])
torch.Size([3, 4, 180])
tensor([1, 0, 0])
800
180


In [49]:
def train(model,device,train_dataloader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (seq, classes) in enumerate(train_dataloader):
        classes=classes.type(torch.LongTensor)
        seq,classes=seq.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)     
        optimizer.zero_grad()
        output=model(seq)
        '''
        print("Batch:", batch_ids)
        print("Output size:", output.size())  
        print("Classes size:", classes.size())
        print("Classes:", classes[0:2])  # Print a few labels
        print("Output:", output[0:2])  # Print a few output predictions
        print(len(output))
        print(len(classes))
        print("classes")
        print(classes[1])
        print("output")
        print(output[0])
        print(output)
        print("length of classes :", len(classes))
        print("Length of output :",len(output))
        '''
        
        
        loss = loss_fn(output,classes)                
        
        loss.backward()
        optimizer.step()
    if(batch_ids +1) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epochs, batch_ids* len(seq), len(train_dataloader.dataset),
            100.*batch_ids / len(train_dataloader),loss.item()))

In [50]:
def test(model, device, test_dataloader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for seq,classes in test_dataloader:
            seq,classes=seq.to(device), classes.to(device)
            y_hat=model(seq)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataloader.dataset)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataloader.dataset),100.*correct/len(test_dataloader.dataset)))
        print("test loss :",test_loss)
        print("Correct :",correct)
        print("Len test dataloader :", len(test_dataloader.dataset))
        print('='*30)

In [51]:
# WE ARE USING RANDOM DATA SO THE TRAINING AND TESTING DOES NOT MATTER, THE AIM IS TO SHOWCASE THE USE OF A CUSTOM DATASET
# SINCE IN PRACTICAL SENSE YOU HAVE TO CLEAN THE DATA AND LOAD THE DATA INTO THE MODEL.


if __name__=='__main__':
    seed=42
    EPOCHS=10
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_dataloader,optimizer,epoch)
        test(model,device,test_dataloader)

inside train
Train Epoch: 1 [2397/2400 (100%)]	Loss: 0.731817

 Test set: Avarage loss: 0,Accuracy:300/600 (50%)

test loss : 0.020114997612933318
Correct : 300
Len test dataloader : 600
inside train
Train Epoch: 2 [2397/2400 (100%)]	Loss: 0.695308

 Test set: Avarage loss: 0,Accuracy:306/600 (51%)

test loss : 0.017732624004905422
Correct : 306
Len test dataloader : 600
inside train
Train Epoch: 3 [2397/2400 (100%)]	Loss: 1.061858

 Test set: Avarage loss: -0,Accuracy:406/600 (68%)

test loss : -0.10431617143253485
Correct : 406
Len test dataloader : 600
inside train
Train Epoch: 4 [2397/2400 (100%)]	Loss: 0.641172

 Test set: Avarage loss: -0,Accuracy:394/600 (66%)

test loss : -0.14636392505529025
Correct : 394
Len test dataloader : 600
inside train
Train Epoch: 5 [2397/2400 (100%)]	Loss: 0.512703

 Test set: Avarage loss: -0,Accuracy:414/600 (69%)

test loss : -0.23939583009729784
Correct : 414
Len test dataloader : 600
inside train
Train Epoch: 6 [2397/2400 (100%)]	Loss: 0.539249
